In [15]:
import numpy as np
import pandas as pd

from math import sqrt
from sklearn.metrics  import mean_squared_error
from sklearn.metrics import pairwise_distances
from sklearn.neighbors import NearestNeighbors

In [16]:
ratings = pd.read_csv('ratings.csv')
ratings.head()


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [17]:
movies = pd.read_csv('movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [18]:
dataset = pd.merge(ratings,movies, on='movieId').drop(['timestamp', 'genres'], axis= 1)
dataset.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)


In [19]:
missing_pivot = dataset.pivot_table(values='rating', index='userId', columns='title')

rate = {}
rows_indexes = {}

for i, row in missing_pivot.iterrows():
    rows = [x for x in range(0,len(missing_pivot.columns))]
    combine = list(zip(row.index, row.values, rows))
    rated = [(x,z) for x,y,z in combine if str(y) != 'nan']
    index = [i[1] for i in rated]
    row_names = [i[0] for i in rated]
    rows_indexes[i] = index
    rate[i] = row_names

missing_pivot.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
pivot_table = dataset.pivot_table(values= 'rating', index= 'userId', columns= 'title').fillna(0)
pivot_table = pivot_table.apply(np.sign)


notrated = {}
notrated_indexes = {}

for i,row in pivot_table.iterrows():
    rows = [x for x in range(0, len(missing_pivot.columns))]
    combine = list(zip(row.index, row.values, row))
    idx_row = [(idx,col) for idx, val, col  in combine if not val > 0]
    indices = [i[1] for i in idx_row]
    row_names = [i[0] for i in idx_row]
    notrated[i] = row_names
    notrated_indexes[i] = indices

pivot_table.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
n = 5
nn = NearestNeighbors(n_neighbors=n, algorithm='brute', metric= 'cosine')
item_nn_fit = nn.fit(pivot_table.T.values)
item_distance, item_indices = item_nn_fit.kneighbors(pivot_table.T.values)

In [22]:
item_dic = {}
for i in range(len(pivot_table.T.index)):
    item_idx = item_indices[i]
    col_names = pivot_table.T.index[item_idx].tolist()
    item_dic[pivot_table.T.index[i]] = col_names

In [23]:
topRecs = {}
for k,v in rows_indexes.items():
    item_idx = [j for i in item_indices[v] for j in  i]
    item_dist = [j for i in item_distance[v] for j in i]
    combine = list(zip(item_dist, item_idx))
    diction = {1:d for d,i in  combine if i not in v}
    zipped = list(zip(diction.keys(), diction.values()))
    sort = sorted(zipped, key=lambda x:x[1])
    recommendations = [(pivot_table.columns[i], d) for i,d in sort]
    topRecs[k] = recommendations

In [32]:
def getrecommendations(user, number_of_recs = 30):
    if user > len(pivot_table.index):
        print('Out of Range, there are only {} users, try number below!'.format(len(pivot_table.index)))
    else:
        print("You watched these movies: \n\n{}".format('\n'.join(rate[user])))
        print()
        print('Recommended Movies:\n')
    for k,v in topRecs.items():
        if user == k:
            for i in v[:number_of_recs]:
                print('{} with similarity: {:.4f}'.format(i[0], 1 - i[1]))

In [34]:
getrecommendations(534)

You watched these movies: 

101 Dalmatians (One Hundred and One Dalmatians) (1961)
10th Kingdom, The (2000)
2 Fast 2 Furious (Fast and the Furious 2, The) (2003)
2012 (2009)
21 Jump Street (2012)
22 Jump Street (2014)
300 (2007)
40-Year-Old Virgin, The (2005)
47 Ronin (2013)
50 First Dates (2004)
A-Team, The (2010)
A.I. Artificial Intelligence (2001)
Absolutely Anything (2015)
Ace Ventura: Pet Detective (1994)
Ace Ventura: When Nature Calls (1995)
Adventures of Tintin, The (2011)
Aeon Flux (2005)
Aladdin (1992)
Amazing Spider-Man, The (2012)
American Beauty (1999)
American History X (1998)
American Pie (1999)
American Pie Presents: Band Camp (American Pie 4: Band Camp) (2005)
Anacleto: Agente secreto (2015)
Anastasia (1997)
Angriest Man in Brooklyn, The (2014)
Animatrix, The (2003)
Ant-Man (2015)
Antz (1998)
Armageddon (1998)
Army of Darkness (1993)
As Good as It Gets (1997)
Asterix & Obelix vs. Caesar (Astérix et Obélix contre César) (1999)
Asterix & Obelix: Mission Cleopatra (Astérix

In [29]:
item_distance = 1 - item_distance
predictions = item_distance.T.dot(pivot_table.T.values)/np.array([np.abs(item_distance.T).sum(axis=1)]).T
ground_truth = pivot_table.T.values[item_distance.argsort()[0]]

def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

error_rate = rmse(predictions, ground_truth)
print("Accuracy: {:.3f}".format(100-error_rate))
print("RMSE: {:.5f}".format(error_rate))

Accuracy: 99.051
RMSE: 0.94922
